In [1]:
import os
import getpass

# LANGSMITH
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LANGSMITH API key: ")

# GROQ
if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

from langchain.chat_models import init_chat_model 
model = init_chat_model("llama3-8b-8192", model_provider="groq")


In [4]:
from langchain_core.messages import HumanMessage, AIMessage

# LangGraph Message Persistence

In [2]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema = MessagesState)

def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

# Define a node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
# this gives an identification to a chat session
config = {"configurable": {"thread_id": "abc123"}}

In [5]:
# invoke
query = "Hi! I'm Bob"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {
        "messages": input_messages
    },
    config=config
)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?


In [6]:
query = "What's my name?"


input_messages = [HumanMessage(query)]
output = app.invoke(
    {
        "messages": input_messages
    },
    config=config
)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob!


In [8]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob!


# Async

In [9]:

# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I apologize, but I'm a large language model, I don't have the ability to know your name or any personal information about you. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations. If you'd like to introduce yourself, I'd be happy to chat with you!


# Prompt Template

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name = "messages")
    ]
)


In [11]:
workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)



In [12]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Arrr, shiver me timbers! 'Tis a pleasure to make yer acquaintance, Jim me lad! Welcome aboard me digital vessel, where I'll be doin' me best to answer yer questions and keep ye entertained with me swashbucklin' ways. What be bringin' ye to these fair shores, matey?


In [13]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Arrr, ye be askin' about yer own name, eh? Well, matey, I be tellin' ye that yer name be Jim! Ye told me yerself, just a moment ago, when ye introduced yerself as Jim. So hoist the colors, me hearty, and remember that ye be Jim, the scourge o' the seven seas... er, I mean, the scourge o' the digital seas!


### fancier prompt template

In [17]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [14]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict 

In [18]:
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

workflow = StateGraph(state_schema=State)

def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


In [19]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "German"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {
        "messages": input_messages,
        "language": language
    },
    config = config,
)

output["messages"][-1].pretty_print()


================================== Ai Message ==================================

Hallo Bob! Wie geht es Ihnen? Ich bin hier, um Ihnen zu helfen und Ihre Fragen zu beantworten.Was möchten Sie wissen oder sprechen?


In [20]:
# no need to specify the language in the same config / thread
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Dein Name ist Bob!


# Managing conversation history

In [25]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens = 35,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human",
)

messages = [
    SystemMessage(content = "you are a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),    
]

trimmer.invoke(messages)

[SystemMessage(content='you are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [26]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [27]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I apologize, but I don't have that information. I'm a new conversation and I don't have any personal information about you, including your name.


In [30]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked: "What math problem did I ask?"


# Streaming

In [31]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
# app.stream
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

|Hi| Todd|!| Here|'s| one|:

|Why| couldn|'t| the| bicycle| stand| up| by| itself|?

|(W|ait| for| it|...)

|Because| it| was| two|-t|ired|!

|Hope| that| made| you| smile|!| Do| you| want| to| hear| another| one|?||